# Simple Python examples

This notebook features some simple examples of diagrams created
in Jupyter notebooks using the dual canvas Python interface and avoiding
injected Javascript.  It also shows how to add interactive controls to
simple diagrams.

# The sine function

Here we plot the `sin(x)` function as x varies from -4 to 4.  Below
we will add interactive controls.

In [ ]:
from jp_doodle import dual_canvas
from IPython.display import display

# Use numpy to get some X values and matching sines
import numpy as np
xs = np.linspace(-4,4,300)
# Compute the sine values for the xs
sins = np.sin(xs)
# pair the (x,sin(x)) values.
pairs = zip(xs, sins)

# Display a canvas which can be saved to an image snapshot.
plot = dual_canvas.SnapshotCanvas("simple_sine.png", width=520, height=320)
plot.display_all()

def draw_sine_curve(plot):
    # plot the points scaled 1 unit to 50 pixels in both x and y dimensions.
    frame = plot.rframe(50, 50)
    
    # add a named background rectangle (used for receiving events later).
    background = frame.frame_rect(-4.2, -1.2, 8.4, 2.4, color="cornsilk", name="background")
    
    # draw the sine curve in green.  It has no name and therefore will not receive events.
    frame.polyline(pairs, "green", lineWidth=3)

    # add axes and some labels, also invisible to events.
    frame.lower_left_axes(-4, -1, 4, 1, max_tick_count=6)
    frame.text(4, 0.1, "X", align="right")
    frame.text(0.1, 1, "Y", valign="center")
    return (frame, background)

(frame, background) = draw_sine_curve(plot)

# fit the plot into the canvas
plot.fit()

Snapshotting the plot shown above produces the following image:

<img src="simple_sine.png" width="520">

**Exercise:**
Change `draw_sine_curve` to also draw a cosine curve over the same
`x` values in cyan.

**Exercise**
Make a similar plot which plots the $y = x^2$ curve as $x$ varies
from -1 to 1.  Do the same for $y = x^2 - x^3$.

# Sine curve with slider

Now we make the same plot as above but add a sliding text and reference line
which update as the mouse moves over the background region.

In [ ]:
# Make a copy of the above diagram
plot2 = dual_canvas.SnapshotCanvas("simple_sine_slider.png", width=520, height=320)
plot2.display_all()
(frame, background) = draw_sine_curve(plot2)

# add a slider line which is invisible to event handling.
slider_line = frame.line(
    x1=-1, y1=-1,
    x2=-1, y2=1, 
    color="blue", name="line", events=False, hide=True, lineWidth=3)

# add a sliding text feedback
slider_text = frame.text(name="text", hide=True, events=False,
    x=0, y=1.1, text="x=0, sin(x)=0", color="blue", align="center", background="#fb9"
    )

# When the mouse moves over the background, slide the slider to the mouse.
def move_slider_to_event(event):
    position = event['model_location']
    x = position["x"]
    # Don't redraw the canvas until both updates are complete
    with frame.delay_redraw():
        slider_line.change(x1=x, x2=x, hide=False)
        slider_text.change(x=x, text="x=%3.2f; sin(x)=%3.2f" % (x, np.sin(x)), hide=False)
background.on("mousemove", move_slider_to_event)

# fit the plot into the canvas
plot2.fit()
#plot2.debugging_display()

Here is a snapshot of the diagram after mousing over the plot.

<img src="simple_sine_slider.png?a=b" width="520">

**Exercise:** Change the callback so the text display aligns
to the right or left when the slider line gets too close to the 
left or right edges of the diagram, but remains aligned to the
center otherwise.

**Exercise:** Split the text control so the `x` value is at the top
of the slider line and the `sin(x)` value is at the bottom.  Update
the callback appropriately.

# Image color sampling

The example below selects a color sample from an image, displays the color, and delivers
the color values to a Python function callback.

In [ ]:
# reading an image from python
import matplotlib.image as mpimg
img = mpimg.imread('./mandrill.png')
img.shape, img.max(), img.min()

In [ ]:
# Scale the inverted image to byte values.
inverted = (1 - img) * 255

# Create the canvas for the color sampler
color_sampler = dual_canvas.SnapshotCanvas("color_sampler.png", width=820, height=320)
color_sampler.display_all()

# Load the image to the canvas and name it.
color_sampler.name_image_array("inverted mandrill", inverted)

# Create a reference frame with y pointing downward with origin at canvas position (0, 200)
frame = color_sampler.rframe(scale_x=1, scale_y=-1, translate_x=0, translate_y=200)

# Draw the full image by name at normal resolution with lower left corner at frame (0, 200)
full = frame.named_image("inverted mandrill", 0, 200, 200, 200, name=True)

# Draw a 3x3 image window to track mouse position.
window = frame.named_image("inverted mandrill", 220, 200, 200, 200, sx=0, sy=0, sWidth=3, sHeight=3, name=True)

# Draw a color swatch
swatch = frame.frame_rect(440, 0, 200, 200, color="pink", name=True)
detail = frame.text(440, 0, text="No color selected yet", name=True)

# Mouse tracker, does not respond to events
tracker = frame.frame_rect(0, 0, 10, 10, dx=-5, dy=-5, color="#999", 
                           fill=False, lineWidth=5, events=False, name=True)

events = []
color_array = []

def track_mouse(event):
    events.append(event)
    position = event['model_location']
    #print "mouse move at", position
    #print event
    x = int(position["x"])
    y = int(position["y"])
    tracker.change(x=x, y=y)
    window.change(sx=x-1, sy=y-1)
    color_array[:] = inverted[y][x]
    (r,g,b) = color_array
    color_name = "rgb(%s,%s,%s)" % (r,g,b)
    detail.change(text="color at (%s,%s) is %s" % (x,y,color_name))
    swatch.change(color=color_name)

full.on("mousemove", track_mouse)
#full.on("click", track_mouse)
#frame.on_canvas_event("click", track_mouse)

color_sampler.fit(None, 20)

A snapshot of the color sampler in action looks like this:

<img src="color_sampler.png" width="500">

In [ ]:
events


In [ ]:
frame.attribute_name